Recycling robot - Find optimal policy and optimal value functions

In [14]:
states = ['high', 'low']
actions = ['search', 'wait', 'recharge']
alpha = 0.6 # search; high -> high
beta = 0.3 # search; low -> low

search_reward = 2
search_depletion_reward = -3
wait_reward = 1
gamma = 0.8 # discount factor

 

In [15]:
def actions(state):
    if state == 'high':
        return ["search", "wait"]
    else:
        return ["search", "wait", "recharge"]

# Transition function: returns a list of (probability, next_state, reward) tuples
def transitions(state, action):
    if state == 'high':
        if action == "search":
            return [
                (alpha, 'high', search_reward),
                (1 - alpha, 'low', search_reward),
            ]
        elif action == "wait":
            return [(1.0, 'high', wait_reward)]

    elif state ==  'low':
        if action == "search":
            return [
                (beta, 'low', search_reward),
                (1 - beta, 'high', search_depletion_reward),
            ]
        elif action == "wait":
            return [(1.0, 'low', wait_reward)]
        elif action == "recharge":
            return [(1.0, 'high', 0)]

In [ ]:
import numpy as np
V = np.zeros(2)
theta = 1e-6

while True:
    delta = 0
    Q = {}
    for idx, s in enumerate(states):
        v = V[idx]
        Q[s] = {}

        for a in actions(s):
            q = 0
            for p, s_next, r in transitions(s, a):
                q += p * (r + gamma * V[states.index(s_next)])
            Q[s][a] = q

        V[idx] = max(Q[s].values())
        delta = max(delta, abs(v - V[idx]))

    if delta < theta:
        break

Q = {}

for idx, s in enumerate(states):
    Q[s] = {}
    for a in actions(s):
        q = 0
        for p, s_next, r in transitions(s, a):
            q += p * (r + gamma * V[states.index(s_next)])
        Q[s][a] = q
        
# Extract optimal policy
policy = {}
for s in states:
    action_values = {}
    for a in actions(s):
        q = 0
        for p, s_next, r in transitions(s, a):
            q += p * (r + gamma * V[states.index(s_next)])
        action_values[a] = q

    policy[s] = max(action_values, key=action_values.get)


In [18]:
print("Optimal Value Function:")
for idx, s in enumerate(states):
    print(f"V*({states[idx]}) = {V[idx]:.4f}")

print("\nOptimal Q-values:")
for state, action_vals in Q.items():
    print(f"\nState: {state}")
    for action, value in action_vals.items():
        print(f"  Q*({state}, {action}) = {value:.4f}")

# Extract optimal policy
print("\nOptimal Policy:")
for state, action_vals in Q.items():
    best_action = max(action_vals, key=action_vals.get)
    print(f"  {state} -> {best_action}")

Optimal Value Function:
V*(high) = 7.5758
V*(low) = 6.0606

Optimal Q-values:

State: high
  Q*(high, search) = 7.5758
  Q*(high, wait) = 7.0606

State: low
  Q*(low, search) = 4.1970
  Q*(low, wait) = 5.8485
  Q*(low, recharge) = 6.0606

Optimal Policy:
  high -> search
  low -> recharge
